#import libraries and package

---



In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import Dataset, Subset, DataLoader, ConcatDataset

from torchvision import transforms
from torchvision.models import resnet34

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from PIL import Image
from copy import deepcopy

import numpy as np
import sys
import os
np.set_printoptions(threshold=sys.maxsize) #needed to print correctly the logs

righe di codice utili per importare da altri jupyter notebook delle funzioni. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/MLDL Group project/File colab/File progetto"
!pip install import-ipynb

import import_ipynb 

/content/drive/.shortcut-targets-by-id/193Yl1PYq1kfsJP1wmYcTLxCWdeY9c7Dx/MLDL Group project/File colab/File progetto
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=b124487917987255ef73b0f04dd1f975fd7aefed0cd44e85e98fdd91b27067e8
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
#retrieve from folder 'data'
%cd "/content/drive/MyDrive/MLDL Group project/File colab/File progetto/data"                          
from DataClean import Cifar100 as datasetManager   # from **nome jupyter notebook** import **classe o metodo da importare**
from Manager import Manager

#retrieve from folder 'models'
%cd "/content/drive/MyDrive/MLDL Group project/File colab/File progetto/models"
from resnet import resnet32
from resnetAnnoCorrente import resnet32 as resnet32new
from LwFMC import LWFoptimized
from iCaRL import iCaRL
from iCaRL import Exemplars
from iCaRL import iCaRL_herd




#retrieve from folder 'logs'
%cd "/content/drive/MyDrive/MLDL Group project/File colab/File progetto/logs"
from Save_logs import save_logs

#retrieve from folder 'logs'
%cd "/content/drive/MyDrive/MLDL Group project/File colab/File progetto/logs"
from Save_logs import save_logs



# return in the main project folder
%cd "/content/drive/MyDrive/MLDL Group project/File colab/File progetto"


/content/drive/.shortcut-targets-by-id/193Yl1PYq1kfsJP1wmYcTLxCWdeY9c7Dx/MLDL Group project/File colab/File progetto/data
importing Jupyter notebook from DataClean.ipynb
importing Jupyter notebook from Manager.ipynb
/content/drive/.shortcut-targets-by-id/193Yl1PYq1kfsJP1wmYcTLxCWdeY9c7Dx/MLDL Group project/File colab/File progetto/models
importing Jupyter notebook from resnet.ipynb
importing Jupyter notebook from resnetAnnoCorrente.ipynb
importing Jupyter notebook from LwFMC.ipynb
importing Jupyter notebook from iCaRL_vanilla.ipynb
/content/drive/.shortcut-targets-by-id/193Yl1PYq1kfsJP1wmYcTLxCWdeY9c7Dx/MLDL Group project/File colab/File progetto/logs
importing Jupyter notebook from Save_logs.ipynb
/content/drive/.shortcut-targets-by-id/193Yl1PYq1kfsJP1wmYcTLxCWdeY9c7Dx/MLDL Group project/File colab/File progetto/logs
/content/drive/.shortcut-targets-by-id/193Yl1PYq1kfsJP1wmYcTLxCWdeY9c7Dx/MLDL Group project/File colab/File progetto


utile quando trasforemero tutto in .py, ad ora non serve eseguire:

In [ ]:
# #righe utili ad importare file dalle cartelle logs, models, utils
# file_location = "/content/drive/MyDrive/MLDL Group project/File colab/File progetto/logs/"
# sys.path.append(os.path.abspath(file_location))

# file_location = "/content/drive/MyDrive/MLDL Group project/File colab/File progetto/models/"
# sys.path.append(os.path.abspath(file_location))

# #elimina gli as se tutto funziona come dovrebbe
# from Save_logs import save_logs
# from resnet import resnet32 as resnetPy



# Arguments

In [ ]:
DEVICE=torch.device("cuda:0")
DIR='/content/data'

RANDOM_SEED = [1993,30, 423]    
NUM_CLASSES = 100       # Total number of classes

# Training
BATCH_SIZE = 64         # Batch size 
NUM_EPOCHS = 70         # Total number of training epochs
LR = 2                  # Initial learning rate
MOMENTUM = 0.9          # Momentum for stochastic gradient descent (SGD)
WEIGHT_DECAY = 1e-5     # Weight decay from iCaRL
MILESTONES = [49, 63]   # Step down policy from iCaRL (MultiStepLR)
                        # Decrease the learning rate by gamma at each milestone
GAMMA = 0.2             # Gamma factor from iCaRL


### define dataset transformation ###
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),  #useful data augmentation
                                      transforms.RandomHorizontalFlip(),     #useful data augmentation
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

# Fine tuning implementation
we need to train on 10 new classes and then test on all the seen classes. 
 


In [ ]:
logs = [[] for _ in range(len(RANDOM_SEED))]
RUN_NAME = 'fine_tuning'

for run_i in range(len(RANDOM_SEED)):
  #set the random seed
  random_seed=RANDOM_SEED[run_i]

  #instantiate the net and loss:
  net=resnet32new()
  criterion=nn.BCEWithLogitsLoss()
 
  train_dataset = datasetManager(DIR, train=True, download=True, random_state=random_seed, transform=train_transform)
  test_dataset = datasetManager(DIR, train=False, download=False, random_state=random_seed, transform=test_transform)


  for split_i in range(10):
    print(f"-- Split {split_i} of run {run_i} --")
    
      
      #---- net parameters part ----
    parameters_to_optimize = net.parameters()
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA)

      #---- dataset preparation part ----
    
    #we create the splitted version of train and test:
    ## 10 classes in the train (from current split), 10*(numberOfSeenSplit) for test
    train_dataset.set_classes_batch(train_dataset.batch_splits[split_i])
    test_dataset.set_classes_batch([test_dataset.batch_splits[i] for i in range(0, split_i+1)])

    #instantiate the dataloaders:
    #train and validation split 
    train_idx, val_idx = train_test_split(list(range(len(train_dataset))),      #we extract a split of indices
                                          random_state=random_seed,
                                          test_size = 0.1)   #choose here the test size

    train_data_split = Subset(train_dataset , train_idx)                        #we use the indices previously extracted to make subset of 
    val_data_split = Subset(train_dataset , val_idx)                            #the original train_dataset

    
    train_dataloader=DataLoader(train_data_split, batch_size=BATCH_SIZE,        #we instantiate the dataloaders
                                shuffle=True, num_workers=4, drop_last=True)
    val_dataloader=DataLoader(val_data_split, batch_size=BATCH_SIZE, 
                              shuffle=True, num_workers=4, drop_last=True)

    
    #test dataloader with all seen classes until current iteration:
    test_dataloader=DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)


      #---- end of data preparation ----

      #---- model training ----
    manager = Manager(DEVICE, net, criterion, optimizer, scheduler,
                          train_dataloader,
                          val_dataloader,
                          test_dataloader)
    
    manager.train(NUM_EPOCHS)  # train the model
    
    # Test the model on classes seen until now
    test_accuracy, all_targets, all_preds = manager.test()

    #save test results
    logs[run_i][split_i]['test_accuracy'] = test_accuracy
    logs[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))

    # Add 10 nodes to last FC layer
    manager.increment_classes(n=10)


save_logs(logs,RANDOM_SEED, NUM_EPOCHS, RUN_NAME, BATCH_SIZE)
  


Extracting /content/data/cifar-100-python.tar.gz to /content/data
## Split 0 of run 0 ##


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Output streaming troncato alle ultime 5000 righe.

Epoch: 47, LR: [2]
Train loss: 0.00895928196342928, Train accuracy: 0.9566964285714286
Validation loss: 0.034776919920529635, Validation accuracy: 0.8482142857142857

Epoch: 48, LR: [2]
Train loss: 0.0075644868276348075, Train accuracy: 0.9651785714285714
Validation loss: 0.04667566078049796, Validation accuracy: 0.7991071428571429

Epoch: 49, LR: [2]
Train loss: 0.00761702945788524, Train accuracy: 0.9618303571428571
Validation loss: 0.03513439531837191, Validation accuracy: 0.8415178571428571

Epoch: 50, LR: [0.4]
Train loss: 0.004629490467985826, Train accuracy: 0.9799107142857143
Validation loss: 0.0208493299516184, Validation accuracy: 0.9151785714285714
Best model updated

Epoch: 51, LR: [0.4]
Train loss: 0.001937579653791285, Train accuracy: 0.9962053571428572
Validation loss: 0.017926170889820372, Validation accuracy: 0.9040178571428571

Epoch: 52, LR: [0.4]
Train loss: 0.001570303233789413, Train accuracy: 0.996875
Validation 

#Learning Without Forgetting

In [ ]:
RUN_NAME = 'lwf'

CUDA_LAUNCH_BLOCKING=1
logs = [[] for _ in range(len(RANDOM_SEED))]

for run_i in range(len(RANDOM_SEED)):
  #set the random seed
  random_seed=RANDOM_SEED[run_i]

  #instantiate the net and loss:
  net=resnet32()
  criterion= nn.BCEWithLogitsLoss()



  #load the Cifar100. For each run 'random_seed' makes dataset splits different 
  if run_i== 0: 
    first_run = True         #we download dataset from internet into directory only at the very first instantiation
  else: first_run = False    # questa cosa ho notato la gestisce colab in automatico, é eliminabile

  train_dataset = datasetManager(DIR, train=True, download=first_run, random_state=random_seed, transform=train_transform)
  test_dataset = datasetManager(DIR, train=False, download=False, random_state=random_seed, transform=test_transform)


  for split_i in range(10):
      
    #---- net parameters part ----
    parameters_to_optimize = net.parameters()
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA)

    #---- dataset preparation part ----
    
    #we create the splitted version of train and test:
    ## 10 classes in the train (from current split), 10*(numberOfSeenSplit) for test
    train_dataset.set_classes_batch(train_dataset.batch_splits[split_i])
    test_dataset.set_classes_batch([test_dataset.batch_splits[i] for i in range(0, split_i+1)])

    #instantiate the dataloaders:
    #train and validation split 
    train_idx, val_idx = train_test_split(list(range(len(train_dataset))),      #we extract a split of indices
                                          random_state=random_seed,
                                          test_size = 0.1)   #choose here the test size

    train_data_split = Subset(train_dataset , train_idx)                        #we use the indices previously extracted to make subset of 
    val_data_split = Subset(train_dataset , val_idx)                            #the original train_dataset

    
    train_dataloader=DataLoader(train_data_split, batch_size=BATCH_SIZE,        #we instantiate the dataloaders
                                shuffle=True, num_workers=4, drop_last=True)
    val_dataloader=DataLoader(val_data_split, batch_size=BATCH_SIZE, 
                              shuffle=True, num_workers=4, drop_last=True)

    
    #test dataloader with all seen classes until current iteration:
    test_dataloader=DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)


    #---- end of data preparation ----

    #---- model training ----
    num_classes = 10*(split_i+1)

    if num_classes == 10: # old network == None , we are on first split
                lwf = LWFoptimized(DEVICE, net, None, criterion, optimizer, scheduler,
                            train_dataloader,
                            val_dataloader,
                            test_dataloader,
                            num_classes)
    else:
                  lwf = LWFoptimized(DEVICE, net, old_net, criterion, optimizer, scheduler,
                            train_dataloader,
                            val_dataloader,
                            test_dataloader,
                            num_classes)


    
    lwf.train(NUM_EPOCHS)  # train the model
   
    
    # score record part
    ##saving score for each run and for each split_i
    logs[run_i].append({})
    
    # Test the model on classes seen until now
    test_accuracy, all_targets, all_preds = lwf.test()

    logs[run_i][split_i]['test_accuracy'] = test_accuracy
    logs[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))
    
    old_net = deepcopy(lwf.net)

    lwf.increment_classes()    


save_logs(logs,RANDOM_SEED, NUM_EPOCHS, RUN_NAME, BATCH_SIZE)



# iCaRL (with random exemplar set construction)
iCaRL with random exemplar set construction. Used as comparision to herding strategy.

In [ ]:
RUN_NAME = 'iCarl_no_herding_3run'

CUDA_LAUNCH_BLOCKING=1
NUM_RUNS = len(RANDOM_SEED)


# Initialize logs
logs_icarl = [[] for _ in range(NUM_RUNS)]

for run_i in range(NUM_RUNS):

  train_dataset = datasetManager(DIR, train=True, download=True, random_state=RANDOM_SEED[run_i], transform=train_transform)
  test_dataset = datasetManager(DIR, train=False, download=False, random_state=RANDOM_SEED[run_i], transform=test_transform)
  
  net = resnet32()
  icarl = iCaRL(DEVICE, net, LR, MOMENTUM, WEIGHT_DECAY, MILESTONES, GAMMA, NUM_EPOCHS, BATCH_SIZE, train_transform, test_transform)
  
  for split_i in range(10):
        print(f"## Split {split_i} of run {run_i} ##")


        train_dataset.set_classes_batch(train_dataset.batch_splits[split_i])
        test_dataset.set_classes_batch([test_dataset.batch_splits[i] for i in range(0, split_i+1)])
        

        train_idx, val_idx = train_test_split(list(range(len(train_dataset))),
                                          random_state=RANDOM_SEED[run_i],
                                          test_size = 0.1)   
                                          
        train_data_split = Subset(train_dataset , train_idx)                 #we use the indices previously extracted to make subset of 
        val_data_split = Subset(train_dataset , val_idx)                     #the original train_dataset

    
        #N.B.: Dataloader are instantiated inside iCaRL model
        icarl.incremental_train(split_i, train_data_split, val_data_split)

        #create logs for the current run
        logs_icarl[run_i].append({})

        acc, all_targets, all_preds = icarl.test(test_dataset, train_data_split)

        logs_icarl[run_i][split_i]['accuracy'] = acc
        logs_icarl[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))

save_logs(logs_icarl,RANDOM_SEED, NUM_EPOCHS, RUN_NAME, BATCH_SIZE)

Files already downloaded and verified
## Split 0 of run 0 ##
Length of exemplars set: 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


NameError: ignored

#iCaRL
official iCaRL as in the paper, with herding as construction strategy for exemplar set.

In [ ]:
RUN_NAME = 'iCaRL_herding'
CUDA_LAUNCH_BLOCKING=1
NUM_RUNS = len(RANDOM_SEED)

# Initialize logs
logs_icarl_herd = [[] for _ in range(NUM_RUNS)]

for run_i in range(NUM_RUNS):

  train_dataset = datasetManager(DIR, train=True, download=True, random_state=RANDOM_SEED[run_i], transform=train_transform)
  test_dataset = datasetManager(DIR, train=False, download=False, random_state=RANDOM_SEED[run_i], transform=test_transform)

    
  net = resnet32()
  icarl = iCaRL_herd(DEVICE, net, LR, MOMENTUM, WEIGHT_DECAY, MILESTONES, GAMMA, NUM_EPOCHS, BATCH_SIZE, train_transform, test_transform)

  for split_i in range(10):
        print(f"## Split {split_i} of run {run_i} ##")


        train_dataset.set_classes_batch(train_dataset.batch_splits[split_i])
        test_dataset.set_classes_batch([test_dataset.batch_splits[i] for i in range(0, split_i+1)])
        

        train_idx, val_idx = train_test_split(list(range(len(train_dataset))),      #we extract a split of indices
                                          random_state=RANDOM_SEED[run_i],
                                          test_size = 0.1)   
                                          
        train_data_split = Subset(train_dataset , train_idx)             #we use the indices previously extracted to make subset of 
        val_data_split = Subset(train_dataset , val_idx)                 #the original train_dataset

        #N.B.: Dataloader are instantiated inside iCaRL model
        icarl.incremental_train(split_i, train_data_split, val_data_split)

        logs_icarl_herd[run_i].append({})

        acc, all_targets, all_preds = icarl.test(test_dataset, train_data_split)

        logs_icarl_herd[run_i][split_i]['accuracy'] = acc
        logs_icarl_herd[run_i][split_i]['conf_mat'] = confusion_matrix(all_targets.to('cpu'), all_preds.to('cpu'))


save_logs(logs_icarl_herd,RANDOM_SEED, NUM_EPOCHS, RUN_NAME, BATCH_SIZE)    